In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import argparse
import os
import random
from torch.autograd import Variable
from torch.utils.data import DataLoader
import utils
import numpy as np
from scipy.ndimage.filters import gaussian_filter

In [7]:
# ---------------- load the models  ----------------
tmp = torch.load('./logs/0121_swapCP/smmnist-2/model=dcgan64x64-rnn_size=256-predictor-posterior-rnn_layers=2-1-n_past=5-n_future=10-lr=0.0002-g_dim=128-z_dim=50-last_frame_skip=0-beta=0.0001000/model.pth')
frame_predictor = tmp['frame_predictor']
posterior = tmp['posterior']
content_lstm = tmp['content_lstm']
frame_predictor.eval()
content_lstm.eval()
posterior.eval()
encoder_c = tmp['encoder_c']
encoder_p = tmp['encoder_p']
decoder = tmp['decoder']
encoder_c.train()
encoder_p.train()
decoder.train()

decoder(
  (upc1): Sequential(
    (0): ConvTranspose2d(178, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace)
  )
  (upc2): dcgan_upconv(
    (main): Sequential(
      (0): ConvTranspose2d(1024, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace)
    )
  )
  (upc3): dcgan_upconv(
    (main): Sequential(
      (0): ConvTranspose2d(512, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2, inplace)
    )
  )
  (upc4): dcgan_upconv(
    (main): Sequential(
      (0): ConvTranspose2d(256, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentu

In [10]:
os.mkdir('./figure')
torch.save(decoder, './figure/decoder.pth')
torch.save(encoder_c, './figure/encoder_c.pth')
torch.save(encoder_p, './figure/encoder_p.pth')
torch.save(content_lstm, './figure/content_lstm.pth')
torch.save(frame_predictor, './figure/frame_predictor.pth')
torch.save(posterior, './figure/posterior.pth')